In [1]:
# read /home/s94zalek_hpc/shape_matching/data/data_transmatch/datasets/shrec19/SHREC19_matching_humans/mat/1.mat

import scipy.io

mat = scipy.io.loadmat('/home/s94zalek_hpc/shape_matching/data/data_transmatch/datasets/shrec19/SHREC19_matching_humans/mat/1.mat')

verts = mat['Shape_df'][0][0][0]
faces = mat['Shape_df'][0][0][1]
corr = mat['smpl_matches']

print('verts.shape:', verts.shape)
print('faces.shape:', faces.shape)
print('corr.shape:', corr.shape)

verts.shape: (4706, 3)
faces.shape: (9408, 3)
corr.shape: (6890, 1)


In [2]:
corr

array([[3043],
       [3044],
       [2943],
       ...,
       [4311],
       [4311],
       [4311]], dtype=uint16)

In [17]:
print(corr.min(), corr.max())

1 4706


In [17]:
import os
import numpy as np
import trimesh
from tqdm import tqdm

mat_dir = '/home/s94zalek_hpc/shape_matching/data/data_transmatch/datasets/shrec19/SHREC19_matching_humans/mat/'
off_dir = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/SHREC19_original/off'
corr_dir = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/SHREC19_original/corres'

ply_dir = '/home/s94zalek_hpc/shape_matching/data/data_transmatch/datasets/shrec19/origin_shrec/mat'

for file in tqdm(sorted(os.listdir(mat_dir))):
    if file.endswith('.mat'):
        mat = scipy.io.loadmat(mat_dir + file)

        # verts = mat['Shape_df'][0][0][0]
        # faces = mat['Shape_df'][0][0][1]
        corr = mat['smpl_matches'].flatten()

        # mesh = trimesh.Trimesh(vertices=verts, faces=faces, process=False)
        mesh = trimesh.load_mesh(ply_dir + '/' + file[:-4] + '.ply')
        mesh.export(off_dir + '/' + file[:-4] + '.off')
        
        
        np.savetxt(corr_dir + '/' + file[:-4] + '.txt', corr, fmt='%d')
        # with open(corr_dir + '/' + file[:-4] + '.vts', 'w') as f:
        #     np.savetxt(f, corr, fmt='%d')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:09<00:00,  4.55it/s]


In [7]:
import trimesh

scene = trimesh.Scene()

In [ ]:
# read tr_reg_000.off from each folder and plot it
# load the mesh from the original folder

scene.geometry.clear()

shrec19_orig_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/SHREC19_original/off'
shrec19_remeshed_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/SHREC19_r/off'

mesh_orig = trimesh.load(shrec19_orig_folder + '/12.off', process=False)
# mesh_orig.faces -= 1
# mesh_orig = trimesh.load('/home/s94zalek_hpc/shape_matching/data/data_transmatch/datasets/shrec19/origin_shrec/mat/12.ply')
scene.add_geometry(mesh_orig)

# load the mesh from the remeshed folder
mesh_remeshed = trimesh.load(shrec19_remeshed_folder + '/12.off', process=False)
mesh_remeshed.vertices += [1, 0, 0]  # move the remeshed mesh to the right
scene.add_geometry(mesh_remeshed)

print(mesh_orig)
print(mesh_remeshed)

scene.show()